# Text classification

You have a lot of options when trying to classify text. In this guide I'll demonstrate a decent amount of techniques for classifying text in four datasets.

1. whether IMDB reviews are positive or negative
2. whether a baby name is male or female
3. what category a newsgroup post belongs to
4. assign one or more labels to Reuters newswire

We'll try a few techniques on these.

* scikit-learn bag-of-words models
    * binary counting (is there word there or not?)
    * interlude: visualizing model parameters
    * td-idf, lemmatization, and hashing: better than binary
* word2vec embeddings
    * embeddings trained in situ
    * using large pre-trained embedding models
* neural networks
    * fasttext-style models
    * short example of recurrent networks

In [1]:
# load various models from scikit-learn's library
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

# scikit-leanr preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier

# also get some metrics to try
from sklearn.metrics import accuracy_score

# get data from scikit-learn
from sklearn.datasets import fetch_20newsgroups

# vectorizing text with scikit-learn
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

# a library for visualizing text classification results
import eli5

# regular expressions in python
import re

# the numpy library for dealing with arrays
import numpy as np

# gensim is a word embedding library
from gensim.models.word2vec import Word2Vec, LineSentence
from gensim.models import KeyedVectors

# keras is an easy-to-use neural network library
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.datasets import imdb
from keras.preprocessing import sequence

# the natural language toolkit provides data and some nlp processing
import nltk
from nltk.corpus import reuters
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

# for timing python code
import timeit

# quickly running a count on a python list
from collections import Counter

Using TensorFlow backend.


## Getting data

### IMDB reviews sentiment analysis

This is a neural network ready dataset from [Keras](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification). The words in the dataset have already been converted into integer IDs, so you can't easily have a look at what's in there.

This is a sentiment analysis or polarity dataset, which means that the target labels are positive or negative. It's a relatively simpler task for a ML model to solve.

I'll be using dictionaries to store my data. After I've grabbed the data from keras, I join the integer IDs with spaces to make text for scikit-learn: scikit-learn's vectorizers expect strings.

In [2]:
imdb_data = {"name" : "imdb", "ovr" : False}
(a, b), (c, d) = imdb.load_data(num_words=50000)
imdb_data["X_train_ids"], imdb_data["y_train"], imdb_data["X_test_ids"], imdb_data["y_test"] = a, b, c, d

# For scikit-learn to like the input data, it will needs strings
imdb_data["X_train"] = [" ".join([str(x) for x in line]) for line in imdb_data["X_train_ids"]]
imdb_data["X_test"] = [" ".join([str(x) for x in line]) for line in imdb_data["X_test_ids"]]

Here's a look at what we're dealing with.

In [3]:
imdb_data["train_size"], imdb_data["test_size"] = len(imdb_data["X_train"]), len(imdb_data["X_test"])
imdb_data["avg_length"] = sum([len(i) for i in imdb_data["X_train_ids"]])/len(imdb_data["X_train_ids"])
imdb_data["vocab_size"] = len(set([i for j in imdb_data["X_train_ids"] for i in j]))

print(f"Observations in training data: {imdb_data['train_size']}; test data: {imdb_data['test_size']}")
print(f"Min number of words per line in training set: {min([len(i) for i in imdb_data['X_train_ids']])}")
print(f"Max number of words per line in training set: {max([len(i) for i in imdb_data['X_train_ids']])}")
print(f"Average number of words per line in training set: {imdb_data['avg_length']}")
print(f"Total vocabulary size: {imdb_data['vocab_size']}")

Observations in training data: 25000; test data: 25000
Min number of words per line in training set: 11
Max number of words per line in training set: 2494
Average number of words per line in training set: 238.71364
Total vocabulary size: 49998


### A lot of baby names

The US government has made available [baby names](https://catalog.data.gov/dataset/baby-names-from-social-security-card-applications-national-level-data) from social security card applications. These records go back to 1880 and also indicate the sex of the baby. I'll be trying to predict which names are male and which are female.

Once you've extracted the files to a folder, the following Python code will join them all into a csv file.

```
import os
import re

with open("babies.csv", "w") as w:
    for f in [f for f in os.listdir(os.getcwd()) if "txt" in str(f)]:
        with open(f) as f:
            year = re.search(r'[\d]{4}', f.name)[0]
            for line in f:
                w.write(year+","+line)
```

What I want to do is sort the names by year, remove all duplicates, and then split older and newer names into the training and test sets, respectively. This way, the test set will tell me if I'm correctly inferring newer names from older names.

Below you can see a sample of a few names.

In [4]:
with open("babies.csv") as f:
    baby_list = f.readlines()

# Sort by first 4 characters, the year
baby_list.sort(key=lambda x: x[:4])

print(baby_list[:5])

['1880,Mary,F,7065\n', '1880,Anna,F,2604\n', '1880,Emma,F,2003\n', '1880,Elizabeth,F,1939\n', '1880,Minnie,F,1746\n']


What I'm going to do is keep every combination of name and sex in a `set()`. Python sets do not keep duplicates and are very fast at performing `if x in y` operations, making them perfect for this work. With the names properly organized, it's easy to keep 20,000 for the test set.

In [5]:
baby_set = set()
unique_baby_list = []

for baby in baby_list:
    if " ".join(baby.split(",")[1:3]) in baby_set:
        pass
    else:
        baby_set.add(" ".join(baby.split(",")[1:3]))
        unique_baby_list.append(baby)

# Pick test set
baby_train = unique_baby_list[:-20000]
baby_test = unique_baby_list[-20000:]

baby_data = {"name" : "baby", "ovr" : False}
baby_data["X_train"] = [baby.split(",")[1] for baby in baby_train]
baby_data["X_test"] = [baby.split(",")[1] for baby in baby_test]
baby_data["y_train"] = [baby.split(",")[2] == "M" for baby in baby_train]
baby_data["y_test"] = [baby.split(",")[2] == "M" for baby in baby_test]

Since the data is split by character, we have a small vocabulary size. Even with the removal of duplicates, we still have 100,000+ records.

In [6]:
baby_data["train_size"], baby_data["test_size"] = len(baby_data["X_train"]), len(baby_data["X_test"])
baby_data["avg_length"] = sum([len(i) for i in baby_data["X_train"]])/len(baby_data["X_train"])
baby_data["vocab_size"] = len(set([i for j in baby_data["X_train"] for i in j]))

print(f"Observations in training data: {baby_data['train_size']}; test data: {len(baby_data['X_test'])}")
print(f"Min number of words per line in training set: {min([len(i) for i in baby_data['X_train']])}")
print(f"Max number of words per line in training set: {max([len(i) for i in baby_data['X_train']])}")
print(f"Average number of characters per name in training set: {baby_data['avg_length']}")
print(f"Total character vocabulary size: {baby_data['vocab_size']}")

Observations in training data: 87973; test data: 20000
Min number of words per line in training set: 2
Max number of words per line in training set: 15
Average number of characters per name in training set: 6.442374364861946
Total character vocabulary size: 52


### Newsgroup posts

These are categorized newsgroup posts you can get [from scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) ([user guide](scikit-learn.org/stable/datasets/twenty_newsgroups.html). These are rather long and varied texts drawn from 18,000 posts. Each of these belong in a different topic. You can read a bit more about the dataset [here](http://qwone.com/~jason/20Newsgroups/).

In [7]:
ng_train_raw = fetch_20newsgroups(subset="train", remove=("headers", "footers", "quotes"))
ng_test_raw = fetch_20newsgroups(subset="test", remove=("headers", "footers", "quotes"))

Since this is a scikit-learn datasource, there are extras you can play with. For example, the target labels can be accessed this way:

In [8]:
print(ng_train_raw.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


An example post looks like this. As you can see, these are multiple sentences.

In [9]:
print(ng_train_raw.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [10]:
ng_data = {"name" : "newsgroup20", "ovr" : False}
ng_data.update({"X_train" : ng_train_raw.data, "y_train" : ng_train_raw.target})
ng_data.update({"X_test" : ng_test_raw.data, "y_test" : ng_test_raw.target})

On average these posts are shorter than the imdb reviews; however, there are some monster posts lurking in there.

The total vocabulary size of the newsgroup set is **much higher** than the others.

In [11]:
ng_data["train_size"], ng_data["test_size"] = len(ng_data["X_train"]), len(ng_data["X_test"])
ng_data["avg_length"] = sum([len(i.split(' ')) for i in ng_data["X_train"]])/len(ng_data["X_train"])
ng_data["vocab_size"] = len(set([i.lower() for j in ng_data["X_train"] for i in j.split(" ")]))

print(f"Observations in training data: {len(ng_data['X_train'])}; test data: {len(ng_data['X_test'])}")
print(f"Min number of words per line in training set: {min([len(i.split(' ')) for i in ng_data['X_train']])}")
print(f"Max number of words per line in training set: {max([len(i.split(' ')) for i in ng_data['X_train']])}")
print(f"Average number of words per line in training set: {ng_data['avg_length']}")
print(f"Total vocabulary size: {ng_data['vocab_size']}")

Observations in training data: 11314; test data: 7532
Min number of words per line in training set: 1
Max number of words per line in training set: 20083
Average number of words per line in training set: 206.15980201520242
Total vocabulary size: 282099


### Reuters newswire dataset

The Reuters dataset is a collection of short categorized news stories. I followed [Martin Thoma's blog post to get started](https://martin-thoma.com/nlp-reuters/).

We're usign the nltk version of the dataset, but I'm not sure what that is exactly. Our dataset has 14333 records, but the more popular [reuters-21578](https://archive.ics.uci.edu/ml/datasets/reuters-21578+text+categorization+collection) has 21578. Since that dataset was collected from 1987 newswire texts, I assume the one we're using is similar.

To get a copy of the Reuters data, you have to use `nltk.download("reuters")`.

In [12]:
def load_reuters():
    reuters_data = {"name" : "reuters", "ovr" : True}
    
    # The test and train sets are listed as IDs in the .fileids() member
    train_ids = list(filter(lambda x: x[:5] == "train", reuters.fileids()))
    test_ids = list(filter(lambda x: x[:4] == "test", reuters.fileids()))
    reuters_data["X_train"] = list(map(lambda x: reuters.raw(x), train_ids))
    reuters_data["X_test"] = list(map(lambda x: reuters.raw(x), test_ids))
    
    # The MultiLabelBinarizer will get you the 1s and 0s your model wants
    mlb = MultiLabelBinarizer(sparse_output=True)
    reuters_data["y_train"] = mlb.fit_transform(list(map(lambda x: reuters.categories(x), train_ids)))
    reuters_data["y_test"] = mlb.transform(list(map(lambda x: reuters.categories(x), test_ids)))
    
    return reuters_data
    
reuters_data = load_reuters()

The main challenge with the Reuters dataset are its large amount of classes and their multi-label nature. Models have to cope with these news items belonging to more than one category.

In [13]:
print(f"Example observation targets: {reuters.categories('test/14832')}")
print(f"Number of classes: {len(reuters.categories())}")
print(reuters.categories())

Example observation targets: ['corn', 'grain', 'rice', 'rubber', 'sugar', 'tin', 'trade']
Number of classes: 90
['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr', 'dmk', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-oil', 'heat', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-oil', 'livestock', 'lumber', 'meal-feed', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-oil', 'palmkernel', 'pet-chem', 'platinum', 'potato', 'propane', 'rand', 'rape-oil', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'rye', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'tin', 'trade', 'veg-oil', 'wheat', 

Most of the observations only have one label.

In [14]:
print(f"Min number of target labels: {min([len(reuters.categories(i)) for i in reuters.fileids()])}")
print(f"Min number of target labels: {max([len(reuters.categories(i)) for i in reuters.fileids()])}")
print(f"Average number of target labels per observation: {sum([len(reuters.categories(i)) for i in reuters.fileids()])/len(reuters.fileids())}")

Min number of target labels: 1
Min number of target labels: 15
Average number of target labels per observation: 1.235446792732666


Although the stats below aren't as high as imdb and newsgroup20, the models will take longer to do the multi-label. I use an `ovr` flag to tell scikit-learn to treat this dataset as a one-vs-rest problem.

In [15]:
reuters_data["train_size"], reuters_data["test_size"] = len(reuters_data["X_train"]), len(reuters_data["X_test"])
reuters_data["avg_length"] = sum([len(i.split(' ')) for i in reuters_data["X_train"]])/len(reuters_data["X_train"])
reuters_data["vocab_size"] = len(set([i.lower() for j in reuters_data["X_train"] for i in j.split(" ")]))

print(f"Observations in training data: {len(ng_data['X_train'])}; test data: {len(reuters_data['X_test'])}")
print(f"Min number of words per line in training set: {min([len(i.split(' ')) for i in reuters_data['X_train']])}")
print(f"Max number of words per line in training set: {max([len(i.split(' ')) for i in reuters_data['X_train']])}")
print(f"Average number of words per line in training set: {sum([len(i.split(' ')) for i in reuters_data['X_train']])/len(reuters_data['X_train'])}")
print(f"Total vocabulary size: {len(set([i.lower() for j in reuters_data['X_train'] for i in j.split(' ')]))}")

Observations in training data: 11314; test data: 3019
Min number of words per line in training set: 3
Max number of words per line in training set: 1571
Average number of words per line in training set: 166.21688763032566
Total vocabulary size: 70000


## Convenience functions

Whenever we want to train on our datasets, we'll have to pre-process them and then train a bunch of different models on them. To do these things I've written some simple functions.

The `vectorize()` functions expects one of scikit-learn's [vectorizers](http://scikit-learn.org/stable/modules/feature_extraction.html) as its first argument, and then vectorizes the two datasets it's given.

In [16]:
def vectorize(vectorizer, x_train, x_test=None):
    train_vec = vectorizer.fit_transform(x_train)
    if x_test:
        test_vec = vectorizer.transform(x_test)
    else:
        test_vec = None
    return train_vec, test_vec

The `model_eval()` function iterates over models and datasets, training and evaluating each one. I had originally included more classification metrics, but I found that evaluating test sets so often can take up a lot of time. I'll stick with test accuracy as my main score.

When you wrap a model in the [`OneVsRestClassifier()`](http://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html) function, it'll be re-run for each label separately. This makes training take a lot more time.

In [17]:
def models_eval(models, datasets, train_key="X_train_vec", test_key="X_test_vec"):
    for dataset in datasets:
        print(f"{dataset['name']:20} train/test {dataset['train_size']}/{dataset['test_size']} total vocab {dataset['vocab_size']}")
        print(f"{20*' '}{57*'-'}")
        results = []
        for name, model in models.items():
            if dataset["ovr"]: model = OneVsRestClassifier(model)
            timer = timeit.default_timer()
            model.fit(dataset[train_key], dataset["y_train"])
            train_elapsed = timeit.default_timer() - timer
            timer = timeit.default_timer()
            train_acc = accuracy_score(y_true = dataset["y_train"], y_pred = model.predict(X=dataset[train_key]))
            test_acc = accuracy_score(y_true = dataset["y_test"], y_pred = model.predict(X=dataset[test_key]))
            eval_elapsed = timeit.default_timer() - timer
            results.append({
                "name" : name, 
                "model" : model, 
                "train_acc" : train_acc, 
                "test_acc" : test_acc, 
                "train_elapsed" : train_elapsed, 
                "eval_elapsed" : eval_elapsed
                })
        results.sort(key=lambda x: -x["test_acc"])
        for result in results:
            print("{:>19} | TRAIN {:5.1f}s | EVAL {:5.1f}s | TRAIN/TEST acc {:4.2f}/{:4.2f} |".format(
                result["name"], 
                result["train_elapsed"], 
                result["eval_elapsed"], 
                result["train_acc"], 
                result["test_acc"]
            ))
        print(20*" "+57*"-")

## Text classification with basic vectorization

We will start off out adventure with the easy-to-use [vectorizers](http://scikit-learn.org/stable/modules/feature_extraction.html) in scikit-learn. Without much effort these will give good results, which shows how useful a well-organized library like scikit-learn is.

My choice of models comes down to whatever will run reasonably fast. I learned about LogisticRegression's `C=` parameter from [Martin Thoma's blog post](https://martin-thoma.com/nlp-reuters/). It's a parameter that's easy to miss in the [scikit-learn documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), but it sometimes gives really nice results.

As I'm starting with word-based vectorization, I won't use the `baby` dataset just yet. I'll need character-level vectorization for that.

In [18]:
list_of_models = {"Logistic" : LogisticRegression(solver="lbfgs", n_jobs = -1), 
                  "Logistic C=1000" : LogisticRegression(solver="lbfgs", n_jobs = -1, C=1000), 
                  "RandomForest 10" : RandomForestClassifier(n_jobs = -1), 
                  "RandomForest 100" : RandomForestClassifier(n_jobs = -1, n_estimators=100), 
                  "RndForest 100 MD25" : RandomForestClassifier(n_jobs = -1, n_estimators=100, max_depth=25), 
                  "DecisionTree" : DecisionTreeClassifier(), 
                  "DecisionTree MD25" : DecisionTreeClassifier(max_depth=25), 
                  "MultinomialNB":MultinomialNB()
                 }

list_of_datasets = [imdb_data, ng_data, reuters_data]

### The simplest bag of words

We'll start with the most simple. If a word is present, it gets a 1; otherwise it gets a 0.

Whenever we test for words and flag them to the models, we're using a technique called "bag of words". Even if we're identifiying short sequences of words, like the presence of "not good" or "red meat", it's still bag of words, or maybe called bag of n-grams. The common alternative is modelling the sequence of words directly, as a kind of time series.

In [19]:
for dataset in list_of_datasets:
    dataset["X_train_vec"], dataset["X_test_vec"] = vectorize(CountVectorizer(max_features=50000, binary=True), dataset["X_train"], dataset["X_test"])

models_eval(list_of_models, list_of_datasets)

imdb                 train/test 25000/25000 total vocab 49998
                    ---------------------------------------------------------
           Logistic | TRAIN   1.8s | EVAL   0.0s | TRAIN/TEST acc 1.00/0.87 |
    Logistic C=1000 | TRAIN   1.6s | EVAL   0.0s | TRAIN/TEST acc 1.00/0.86 |
   RandomForest 100 | TRAIN  12.1s | EVAL   1.2s | TRAIN/TEST acc 1.00/0.85 |
 RndForest 100 MD25 | TRAIN   2.2s | EVAL   0.8s | TRAIN/TEST acc 0.95/0.84 |
      MultinomialNB | TRAIN   0.0s | EVAL   0.0s | TRAIN/TEST acc 0.90/0.83 |
    RandomForest 10 | TRAIN   1.6s | EVAL   0.3s | TRAIN/TEST acc 0.99/0.75 |
  DecisionTree MD25 | TRAIN  10.8s | EVAL   0.0s | TRAIN/TEST acc 0.89/0.73 |
       DecisionTree | TRAIN  23.6s | EVAL   0.0s | TRAIN/TEST acc 1.00/0.71 |
                    ---------------------------------------------------------
newsgroup20          train/test 11314/7532 total vocab 282099
                    ---------------------------------------------------------
           Logisti

### Interlude: the evils of overfitting

If you're like most people, you enjoy nice colorful representations of model output. The [`eli5`](http://eli5.readthedocs.io/en/latest/tutorials/sklearn-text.html) has a few things to help you out there.

In [20]:
vec = CountVectorizer(max_features=50000, binary=True).fit(ng_data["X_train"])

model = LogisticRegression(solver="lbfgs", n_jobs = -1, C=1000)

fit = model.fit(ng_data["X_train_vec"], ng_data["y_train"])

The first entry in the newsgroup20 is about cars, and you can see how the logistic model weighs different words. Words like car, bumper, and engine are all positive.

In [21]:
eli5.show_prediction(fit, ng_train_raw.data[0], vec=vec, target_names=ng_train_raw.target_names, targets=["rec.autos"])

You can view a more compressed version of the information with the `show_weights()` function. For some reason the word `rectum` is the fourth most important for hockey posts.

In [22]:
eli5.show_weights(fit, vec=vec, top=10, target_names=ng_train_raw.target_names, 
                  targets=["rec.autos", "rec.sport.baseball", "rec.sport.hockey", "sci.med", "sci.space"])

While this particular example is a bit crude, it is a case of a model overfitting. The word only appears once in the data as a joke, yet it will influence all future predictions.

In [23]:
wikipedia = "The rectum is the final straight portion of the large intestine."

print(ng_train_raw.target_names[model.predict(vec.transform([wikipedia]))[0]])

rec.sport.hockey


### tdidf with unigrams

Compared to binary counting, here is a better approach: the [Term Document Inverser Document Frequency (TD-IDF) vectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). This vectorizer will count word occurences in a sentence (or "document") but then divide these counts with how frequent each word appears in general. For example, if *aardvark* appears once in a sentence and once in the entire corpus, it gets a count of 1; however, if *and* appears once in a sentence but 10,000 times in the corpus, it gets a TD-IDF count of 0.0001. td-idf normalizes your data in a way that gives an edge to rarer words but a penalty to more common words.

According to [this Wikipedia article](https://en.wikipedia.org/wiki/Tf%E2%80%93idf), 83% of text classification tasks use TD-IDF.

Anyways, I'll keep setting the maximum vocabulary size to 50,000 to make of these examples roughly comparable. Increasing the maximum vocabulary usually increases model accuracy. When scikit-learn is given a max vocabulary size, it'll only keep the most frequent words.

We should get **89%** for imdb, **68%** for newsgroup20, and **80%** for reuters. These are the amounts we'll try to beat afterwards.

In [24]:
for dataset in list_of_datasets:
    dataset["X_train_vec"], dataset["X_test_vec"] = vectorize(TfidfVectorizer(max_features=50000, min_df=5), dataset["X_train"], dataset["X_test"])

models_eval(list_of_models, list_of_datasets)

imdb                 train/test 25000/25000 total vocab 49998
                    ---------------------------------------------------------
           Logistic | TRAIN   0.8s | EVAL   0.0s | TRAIN/TEST acc 0.93/0.89 |
    Logistic C=1000 | TRAIN   1.4s | EVAL   0.0s | TRAIN/TEST acc 1.00/0.86 |
   RandomForest 100 | TRAIN   9.7s | EVAL   1.1s | TRAIN/TEST acc 1.00/0.84 |
 RndForest 100 MD25 | TRAIN   2.7s | EVAL   0.9s | TRAIN/TEST acc 0.97/0.84 |
      MultinomialNB | TRAIN   0.0s | EVAL   0.0s | TRAIN/TEST acc 0.90/0.83 |
    RandomForest 10 | TRAIN   1.4s | EVAL   0.4s | TRAIN/TEST acc 0.99/0.74 |
  DecisionTree MD25 | TRAIN  12.7s | EVAL   0.0s | TRAIN/TEST acc 0.91/0.72 |
       DecisionTree | TRAIN  24.3s | EVAL   0.0s | TRAIN/TEST acc 1.00/0.71 |
                    ---------------------------------------------------------
newsgroup20          train/test 11314/7532 total vocab 282099
                    ---------------------------------------------------------
           Logisti

### tdidf with bigrams

We can help the models by informing them of some short word sequences, say sequences of two: these are called bi-grams. For example the imdb models will get a bit of extra help by knowing the presence of "not good" instead of only "not" and "good" separately; this helps in the sentiment analysis task.

We get a slight increase with with **90%** for imdb but decreases with **67%** for newsgroup20 and **79%** for reuters. Some rarer words are important to the models, and they're being pushed out by more common bi-grams. Increasing the `max_features` to 75,000 doesn't make a lot of difference.

In [25]:
for dataset in list_of_datasets:
    dataset["X_train_vec"], dataset["X_test_vec"] = vectorize(TfidfVectorizer(max_features=50000, ngram_range = [1, 2]), dataset["X_train"], dataset["X_test"])

models_eval(list_of_models, list_of_datasets)

imdb                 train/test 25000/25000 total vocab 49998
                    ---------------------------------------------------------
           Logistic | TRAIN   2.2s | EVAL   0.0s | TRAIN/TEST acc 0.95/0.90 |
    Logistic C=1000 | TRAIN   2.5s | EVAL   0.0s | TRAIN/TEST acc 1.00/0.89 |
      MultinomialNB | TRAIN   0.0s | EVAL   0.0s | TRAIN/TEST acc 0.92/0.88 |
   RandomForest 100 | TRAIN  12.1s | EVAL   1.4s | TRAIN/TEST acc 1.00/0.85 |
 RndForest 100 MD25 | TRAIN   2.8s | EVAL   1.3s | TRAIN/TEST acc 0.96/0.85 |
    RandomForest 10 | TRAIN   1.6s | EVAL   0.6s | TRAIN/TEST acc 0.99/0.76 |
  DecisionTree MD25 | TRAIN  20.5s | EVAL   0.0s | TRAIN/TEST acc 0.91/0.72 |
       DecisionTree | TRAIN  37.7s | EVAL   0.1s | TRAIN/TEST acc 1.00/0.71 |
                    ---------------------------------------------------------
newsgroup20          train/test 11314/7532 total vocab 282099
                    ---------------------------------------------------------
    Logistic C=100

### Pre-processing

Before we move on to other things, we can try pre-processing our text data further. I got the lemmatization code [here](http://scikit-learn.org/stable/modules/feature_extraction.html#customizing-the-vectorizer-classes).

In [26]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

Below you can see that this beats the best we had so far. It won't work with the imdb set. With 75,000 and 1,2 ngrams we get **67%** for newsgroup20 and **80%** for reuters. With only 1,1, we **81%** for reuters, which is out best so far. Newsgroup20 gets **66%**.

In [27]:
list_of_datasets = [ng_data, reuters_data]

In [28]:
for dataset in list_of_datasets:
    dataset["X_train_vec"], dataset["X_test_vec"] = vectorize(TfidfVectorizer(max_features=50000, ngram_range = [1, 1], 
                                                                              tokenizer=LemmaTokenizer(), stop_words="english"), 
                                                              dataset["X_train"], dataset["X_test"])

models_eval(list_of_models, list_of_datasets)

newsgroup20          train/test 11314/7532 total vocab 282099
                    ---------------------------------------------------------
    Logistic C=1000 | TRAIN  16.3s | EVAL   0.0s | TRAIN/TEST acc 0.97/0.66 |
           Logistic | TRAIN   7.3s | EVAL   0.0s | TRAIN/TEST acc 0.87/0.66 |
      MultinomialNB | TRAIN   0.0s | EVAL   0.0s | TRAIN/TEST acc 0.83/0.64 |
   RandomForest 100 | TRAIN   6.5s | EVAL   1.1s | TRAIN/TEST acc 0.97/0.61 |
 RndForest 100 MD25 | TRAIN   0.7s | EVAL   0.7s | TRAIN/TEST acc 0.77/0.58 |
    RandomForest 10 | TRAIN   0.9s | EVAL   0.3s | TRAIN/TEST acc 0.97/0.49 |
       DecisionTree | TRAIN   6.6s | EVAL   0.0s | TRAIN/TEST acc 0.97/0.41 |
  DecisionTree MD25 | TRAIN   2.3s | EVAL   0.0s | TRAIN/TEST acc 0.45/0.31 |
                    ---------------------------------------------------------
reuters              train/test 7769/3019 total vocab 70000
                    ---------------------------------------------------------
    Logistic C=1000 

### hashing trick with character ngrams

So far we've been setting a limit of 50,000 to our vocabulary. What if we wanted to fit more words into that 50,000, perhaps by some form of compression?

The hashing trick hashes words into ID numbers. The same word will be hashed the same way each time, but two words may "collide" and get the same ID number. These collisons are bad for classification, but hashing can turn out to be a net good.

In [29]:
list_of_models = {"Logistic" : LogisticRegression(solver="lbfgs", n_jobs = -1), 
                  "Logistic C=1000" : LogisticRegression(solver="lbfgs", n_jobs = -1, C=1000), 
                  "RandomForest 10" : RandomForestClassifier(n_jobs = -1)
                 }

list_of_datasets = [imdb_data, baby_data, ng_data, reuters_data]

The reason I'm ushing hashing here is to try character n-grams on the baby name dataset. By taking ngrams from 2 to 5, I am going to have a *massive* vocabulary. With hashing it becomes workable.

With the baby dataset, its 1-gram vocabulary of 52 becomes 123,084 with `[2,5]` n-grams. For this we get an accuracy score of **81%**.

In [30]:
for dataset in list_of_datasets:
    dataset["X_train_vec"], dataset["X_test_vec"] = vectorize(HashingVectorizer(n_features = 50000, analyzer="char_wb", ngram_range=[2,5]), 
                                                              dataset["X_train"], dataset["X_test"])

models_eval(list_of_models, list_of_datasets)

imdb                 train/test 25000/25000 total vocab 49998
                    ---------------------------------------------------------
    Logistic C=1000 | TRAIN   7.0s | EVAL   0.1s | TRAIN/TEST acc 0.90/0.86 |
           Logistic | TRAIN   6.0s | EVAL   0.1s | TRAIN/TEST acc 0.81/0.80 |
    RandomForest 10 | TRAIN   2.6s | EVAL   0.7s | TRAIN/TEST acc 0.99/0.69 |
                    ---------------------------------------------------------
baby                 train/test 87973/20000 total vocab 52
                    ---------------------------------------------------------
           Logistic | TRAIN   1.9s | EVAL   0.0s | TRAIN/TEST acc 0.85/0.81 |
    Logistic C=1000 | TRAIN   1.9s | EVAL   0.0s | TRAIN/TEST acc 0.87/0.79 |
    RandomForest 10 | TRAIN  84.7s | EVAL   0.7s | TRAIN/TEST acc 0.90/0.79 |
                    ---------------------------------------------------------
newsgroup20          train/test 11314/7532 total vocab 282099
                    -----------------

## Training word embeddings

In 2015-2017 the internet was infatuated with word embeddings. These little embeddings were able to get the "meaning" of words, and they could find [related vegetables and pokemon](https://fasttext.cc/docs/en/unsupervised-tutorial.html#nearest-neighbor-queries) for you.

Word embeddings are vectors assigned to words, meaning that each word is "embedded" in an n-dimensional space. These embedding spaces can have many dimensions, like 100 to 300. Embedding models organize this space by placing similar words close together: the model decides what words are similar by what other words they co-occur with. For example, if bacon and ham are often mentionned alongside pork, breakfast, and greasy, they are related. This is how word embedding models know how to find related words for you.

Ultimately, these word embeddings give a slight edge to classification models. First, they're in n-dimensional space, where n is 100-300 not 50,000 to 100,000 (total size of vocabulary). This saves resources. Second, models can know beforehand treat ham and bacon similarly.

Word embeddings will work alright with our models, but they won't beat our td-idf champion.

### Embedding our datasets

We can train the vectors easily with the gensim library. There's a little pre-processing involved.

In [31]:
# Lowercase text and remove all non-letters with single spaces
def preprocessor(x):
    return re.sub(r"[ ]+", " ", re.sub(r"[^\w]+", " ", x)).lower()

# Either split words or characters into list items (tokenize)
def w2v_prepare(dataset, by_words=True):
    if by_words:
        return [preprocessor(line).split() for line in dataset]
    else:
        return [list(line) for line in dataset]

# With the tokenized text, run word2vec on it. Afterwards, delete the model and keep the vectors.
def w2v_fit(text, size=100, alpha=0.025, window=5, min_count=5, workers=4, iter=5):
    w2v_model = Word2Vec(text, size=size, alpha=alpha, window=window, min_count=min_count, workers=workers)
    word_vectors = w2v_model.wv
    del w2v_model
    print(f"word2vec model has {len(word_vectors.vocab)} words")
    return word_vectors

# After the vectors are ready, we embed our datasets, averaging afterwards
def w2v_transform(text, word_vectors):
    vocab = set(word_vectors.vocab)
    size = word_vectors.vector_size
    vectorized = []
    for line in text:
        line = list(filter(lambda x: x in vocab, line))
        if line:
            line = np.mean(list(map(lambda x: word_vectors[x], line)), axis=0)
            vectorized.append(line)
        else:
            vectorized.append(np.zeros(size))
    return np.array(vectorized)

Our corpora won't be great for word2vec since they're fairly small. This will hurt the quality of the vectors.

In [32]:
ng_wv = w2v_fit(w2v_prepare(ng_data["X_train"]), min_count=1, iter=50, alpha=0.05)
baby_wv = w2v_fit(w2v_prepare(baby_data["X_train"], by_words=False), size=20)
reuters_wv = w2v_fit(w2v_prepare(reuters_data["X_train"]), min_count=1, iter=50, alpha=0.05)

word2vec model has 101675 words
word2vec model has 52 words
word2vec model has 26319 words


You can query word vectors to get related words. Below I look up "man" in the newsgroup20 embeddings. The matches are reasonable.

In [33]:
ng_wv.most_similar(positive=["man"])

[('woman', 0.7636096477508545),
 ('person', 0.6785249710083008),
 ('child', 0.6541754007339478),
 ('jesus', 0.6385825276374817),
 ('father', 0.6232447624206543),
 ('god', 0.6104214787483215),
 ('mother', 0.6048308610916138),
 ('christ', 0.5973607301712036),
 ('teacher', 0.595134973526001),
 ('lord', 0.5933548212051392)]

The data has to be embedded to be used.

In [34]:
ng_data["X_train_wv"] = w2v_transform(w2v_prepare(ng_data["X_train"]), ng_wv)
ng_data["X_test_wv"] = w2v_transform(w2v_prepare(ng_data["X_test"]), ng_wv)

baby_data["X_train_wv"] = w2v_transform(w2v_prepare(baby_data["X_train"], by_words=False), baby_wv)
baby_data["X_test_wv"] = w2v_transform(w2v_prepare(baby_data["X_test"], by_words=False), baby_wv)

reuters_data["X_train_wv"] = w2v_transform(w2v_prepare(reuters_data["X_train"]), reuters_wv)
reuters_data["X_test_wv"] = w2v_transform(w2v_prepare(reuters_data["X_test"]), reuters_wv)

Since we now have "denser" data (100 dimensions instead of 25,000), I'll try out KNN since it won't take forever anymore (but it will still take a long time). KNN does surprisingly work suprisingly well on the babies and reuters datasets, especially with more neighbors (67% on babies and 74% on reuters).

In [35]:
list_of_models = {"Logistic" : LogisticRegression(solver="lbfgs", n_jobs = -1), 
                  "Logistic C=1000" : LogisticRegression(solver="lbfgs", n_jobs = -1, C=1000), 
                  "RandomForest 10" : RandomForestClassifier(n_jobs = -1), 
                  "RandomForest 100" : RandomForestClassifier(n_jobs = -1, n_estimators=100), 
                  "RandomForest 100/10" : RandomForestClassifier(n_jobs = -1, n_estimators=100, max_depth=10), 
                  "KNN 1" : KNeighborsClassifier(n_neighbors = 1, n_jobs=-1)
                 }

list_of_datasets = [ng_data, baby_data, reuters_data]

In [36]:
models_eval(list_of_models, list_of_datasets, train_key="X_train_wv", test_key="X_test_wv")

newsgroup20          train/test 11314/7532 total vocab 282099
                    ---------------------------------------------------------
    Logistic C=1000 | TRAIN   8.9s | EVAL   0.0s | TRAIN/TEST acc 0.54/0.47 |
           Logistic | TRAIN   9.0s | EVAL   0.0s | TRAIN/TEST acc 0.53/0.47 |
RandomForest 100/10 | TRAIN   1.6s | EVAL   0.5s | TRAIN/TEST acc 0.90/0.40 |
   RandomForest 100 | TRAIN   2.0s | EVAL   0.5s | TRAIN/TEST acc 0.97/0.40 |
              KNN 1 | TRAIN   0.1s | EVAL   4.2s | TRAIN/TEST acc 0.97/0.33 |
    RandomForest 10 | TRAIN   0.4s | EVAL   0.3s | TRAIN/TEST acc 0.97/0.31 |
                    ---------------------------------------------------------
baby                 train/test 87973/20000 total vocab 52
                    ---------------------------------------------------------
   RandomForest 100 | TRAIN   7.3s | EVAL   1.1s | TRAIN/TEST acc 0.87/0.65 |
              KNN 1 | TRAIN   0.1s | EVAL   5.3s | TRAIN/TEST acc 0.90/0.65 |
    Logistic C=1000 |

### Using large pre-trained embeddings

Above we trained our word embedding model on our datasets, but is that a good idea? Are toy datasets for text classification sufficient to model the English language? Remember that word2vec operates on co-occurences and that words can appear in many different contexts. It's tough to infer the meaning of a word from only a few examples.

Therefore a common strategy with word embeddings is ti simply use pre-trained embeddings from a massive dataset. Thanks to the size of the source corpus, the word relationships are bound to be more finely detailed.

You can get a 3.6GB word vector file from [this blogger](http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/) or this [archived Google Code post](https://code.google.com/archive/p/word2vec/)

In [37]:
googlenews = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)
print(f"word2vec model has {len(googlenews.vocab)} words")

word2vec model has 3000000 words


Prepare our data.

In [38]:
ng_data["X_train_wv"] = w2v_transform(w2v_prepare(ng_data["X_train"]), googlenews)
ng_data["X_test_wv"] = w2v_transform(w2v_prepare(ng_data["X_test"]), googlenews)

reuters_data["X_train_wv"] = w2v_transform(w2v_prepare(reuters_data["X_train"]), googlenews)
reuters_data["X_test_wv"] = w2v_transform(w2v_prepare(reuters_data["X_test"]), googlenews)

Again, I'll try the same models as above. I am going to ditch the imdb and babies dataset because they lack actual words. (The imdb data is already coded to integer IDs and the baby names are a character-level task.

In [39]:
list_of_models = {"Logistic" : LogisticRegression(solver="lbfgs", n_jobs = -1), 
                  "Logistic C=1000" : LogisticRegression(solver="lbfgs", n_jobs = -1, C=1000), 
                  "RandomForest 10" : RandomForestClassifier(n_jobs = -1), 
                  "RandomForest 100" : RandomForestClassifier(n_jobs = -1, n_estimators=100), 
                  "RandomForest 100/10" : RandomForestClassifier(n_jobs = -1, n_estimators=100, max_depth=10), 
                  "KNN 1" : KNeighborsClassifier(n_neighbors = 1, n_jobs=-1)
                 }

list_of_datasets = [ng_data, reuters_data]

The results below are not the best, but they do beat our previous word embedding results. KNN 5 again performs well on the reuters data (76% with KNN 5--not shown).

In [40]:
models_eval(list_of_models, [ng_data, reuters_data], train_key="X_train_wv", test_key="X_test_wv")

newsgroup20          train/test 11314/7532 total vocab 282099
                    ---------------------------------------------------------
           Logistic | TRAIN  11.2s | EVAL   0.0s | TRAIN/TEST acc 0.65/0.60 |
    Logistic C=1000 | TRAIN  21.8s | EVAL   0.0s | TRAIN/TEST acc 0.75/0.60 |
RandomForest 100/10 | TRAIN   2.6s | EVAL   0.5s | TRAIN/TEST acc 0.95/0.50 |
   RandomForest 100 | TRAIN   3.2s | EVAL   0.5s | TRAIN/TEST acc 0.97/0.50 |
              KNN 1 | TRAIN   0.1s | EVAL  11.4s | TRAIN/TEST acc 0.97/0.44 |
    RandomForest 10 | TRAIN   0.5s | EVAL   0.2s | TRAIN/TEST acc 0.97/0.33 |
                    ---------------------------------------------------------
reuters              train/test 7769/3019 total vocab 70000
                    ---------------------------------------------------------
              KNN 1 | TRAIN   4.8s | EVAL 228.4s | TRAIN/TEST acc 0.99/0.74 |
    Logistic C=1000 | TRAIN  98.8s | EVAL   0.3s | TRAIN/TEST acc 0.94/0.74 |
RandomForest 100/10 

## Neural networks

We aren't going to be using deep learning here; instead we're just going to use neural networks to train embeddings on-the-fly. Rather than training twice, first on embeddings and then on classification, the word embeddings will be trained alongside classification.

The method we use below is effectively [fasttext](https://fasttext.cc/), a fast barebones classifier. It's pretty fast and tends to meet or beat our best results.

keras expects sequences of integers as inputs. These inputs are fed to embedding layers that do similar work to word2vec above: words get embeddings that are useful for classification.

In [41]:
# given a list of tokens, transform them into ngrams
def get_ngrams(x, n=1):
    if n==1:
        return x
    elif len(x) >= n:
        return ["_".join(x[i:i+n]) for i in range(len(x)-n+1)]
    else:
        return []

# with an ngram interval, such as [2,5], break a list of tokens into the appropriate ngrams
def ngram_iter(x, interval):
    y = []
    for n in range(interval[0], interval[1]+1):
        y += get_ngrams(x, n)
    return y

# prepare data for keras's embeddings layers
# this transforms lists of tokens into
def keras_data(train_set, test_set, by_words=True, max_unigrams=50000, ngram_range=[1,1]):
    train_set = w2v_prepare(train_set, by_words)
    test_set = w2v_prepare(test_set, by_words)
    
    id2word = [i for line in train_set for i in ngram_iter(line, ngram_range)]
    
    if max_unigrams > 0:
        id2word = Counter(id2word)
        id2word = list(id2word.items())
        id2word.sort(key=lambda x: -x[1])
        id2word = [x[0] for x in id2word[:max_unigrams-2]]
        id2word = ["<PAD>", "<NULL>"] + list(set(id2word))
    else:
        id2word = ["<PAD>", "<NULL>"] + list(set(id2word))

    word2id = dict()
    vocab_size = len(id2word)
    print(f"Size of vocabulary: {vocab_size}")
    for i in range(vocab_size):
        word2id[id2word[i]] = i

    train_set = [[word2id.get(token, 1) for token in ngram_iter(line, ngram_range)] for line in train_set]
    test_set = [[word2id.get(token, 1) for token in ngram_iter(line, ngram_range)] for line in test_set]
    
    return train_set, test_set

Through trial and error I've ended up with the settings below.

In [42]:
ng_data["X_train_ids"], ng_data["X_test_ids"] = keras_data(ng_data["X_train"], ng_data["X_test"], ngram_range=[1,1])
baby_data["X_train_ids"], baby_data["X_test_ids"] = keras_data(baby_data["X_train"], baby_data["X_test"], by_words=False, ngram_range=[2,5])
reuters_data["X_train_ids"], reuters_data["X_test_ids"] = keras_data(reuters_data["X_train"], reuters_data["X_test"], ngram_range=[1,2])

Size of vocabulary: 50000
Size of vocabulary: 50000
Size of vocabulary: 50000


### IMDB

The imdb dataset is used in keras's example implementation of [fastext](https://github.com/keras-team/keras/blob/master/examples/imdb_fasttext.py). I've tweaked it a bit to give the best results.

As you can see, the vector size of the embeddings is only *one* and these feed directly to a *single* sigmoid neuron. Nevertheless the model works quite well. *Always try simpler networks!*

In [43]:
x_train = sequence.pad_sequences(imdb_data["X_train_ids"], maxlen=400)
x_test = sequence.pad_sequences(imdb_data["X_test_ids"], maxlen=400)

model = Sequential()
model.add(Embedding(50000, 1, input_length=400))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=0.1)
    
model.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

early_stop = EarlyStopping(min_delta=0.01, patience=2)

model.fit(x_train, imdb_data["y_train"],
          batch_size=256,
          epochs=20,
          callbacks=[early_stop],
          validation_data=(x_test, imdb_data["y_test"]))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 1s 54us/step - loss: 0.4950 - acc: 0.7659 - val_loss: 0.3333 - val_acc: 0.8725
Epoch 2/20
25000/25000 [==============================] - 0s 9us/step - loss: 0.2308 - acc: 0.9177 - val_loss: 0.2969 - val_acc: 0.8780
Epoch 3/20
25000/25000 [==============================] - 0s 9us/step - loss: 0.1473 - acc: 0.9526 - val_loss: 0.2891 - val_acc: 0.8869
Epoch 4/20
25000/25000 [==============================] - 0s 9us/step - loss: 0.0986 - acc: 0.9726 - val_loss: 0.3359 - val_acc: 0.8704


### Newsgroup

The newsgroup20 dataset is a bit more complex, so it requires a few more parameters.

In [44]:
x_train = sequence.pad_sequences(ng_data["X_train_ids"], maxlen=500)
x_test = sequence.pad_sequences(ng_data["X_test_ids"], maxlen=500)

model = Sequential()
model.add(Embedding(50000, 32, input_length=500))
model.add(GlobalAveragePooling1D())
model.add(Dense(len(ng_train_raw.target_names), activation='softmax'))

optimizer = Adam(lr=0.01)
    
model.compile(loss='sparse_categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

early_stop = EarlyStopping(min_delta=0.01, patience=5)

model.fit(x_train, ng_data["y_train"],
          batch_size=64,
          epochs=50,
          callbacks=[early_stop],
          validation_data=(x_test, ng_data["y_test"]))

Train on 11314 samples, validate on 7532 samples
Epoch 1/50
11314/11314 [==============================] - 1s 84us/step - loss: 2.8141 - acc: 0.1640 - val_loss: 2.5467 - val_acc: 0.2797
Epoch 2/50
11314/11314 [==============================] - 1s 46us/step - loss: 2.1241 - acc: 0.4684 - val_loss: 1.9837 - val_acc: 0.4968
Epoch 3/50
11314/11314 [==============================] - 1s 47us/step - loss: 1.5417 - acc: 0.6623 - val_loss: 1.6533 - val_acc: 0.5793
Epoch 4/50
11314/11314 [==============================] - 1s 47us/step - loss: 1.1699 - acc: 0.7678 - val_loss: 1.4782 - val_acc: 0.6158
Epoch 5/50
11314/11314 [==============================] - 1s 45us/step - loss: 0.9278 - acc: 0.8177 - val_loss: 1.3758 - val_acc: 0.6235
Epoch 6/50
11314/11314 [==============================] - 1s 46us/step - loss: 0.7555 - acc: 0.8509 - val_loss: 1.3168 - val_acc: 0.6503
Epoch 7/50
11314/11314 [==============================] - 1s 46us/step - loss: 0.6277 - acc: 0.8768 - val_loss: 1.2627 - val_acc:

### Reuters

If you run the neural network on Reuters with the softmax function, you'll beat the rest with a score of **82%**.

However, this approach makes no sense because we have multi-label problem (one record can have more than one label), not a multi-class problem (one record has only one label). The softmax function outputs probabilities that sum to 1; in the case of accuracy, the highest probability is the chosen label. That score of 82% above just means that the neural network beats the other models with one hand tied behind its back (only being able to predict one class). Why? Most Reuters observations only have one class/label with an average of 1.25 label/obs.

Instead, the proper thing to do is use a sigmoid action function at output so that each label is evaluated separately. The sigmoids will output 0 or 1, and one sigmoid doesn't affect the other, unlike the softmax.

Even though this approach is the correct one, it makes accuracy kind of meaningless. With sigmoids the accuracy shoots to 99% because the neural networks gets a point each time it predicts 0 correctly: the target labels are pretty sparse so this is pretty easy to do! In fact, if the model only predicts 0s, it does pretty well. We need a better metric.

I'm not extremely familiar with precision, recall, and f1, but I think they're the right ones to use. We want to measure around real and predicted positives. [Here is the formula for the f1-score.](https://en.wikipedia.org/wiki/F1_score) Precision goes down whenever a positive label is assigned incorrectly, and recall goes down whenever a true positive label is incorrectly missed. If the model only predicts 0s, its recall score will be zero; if the model only predicts 1s, its precision score will be zero. This sounds good.

keras removed its precision, recall, and f1 metrics, so you need to define them yourself. Or, you could just grab [some code on Stack Overflow](https://stackoverflow.com/a/45305384).

In [45]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

With the f1-score metric, the model has a test score of about 81%.

In [46]:
x_train = sequence.pad_sequences(reuters_data["X_train_ids"], maxlen=500)
x_test = sequence.pad_sequences(reuters_data["X_test_ids"], maxlen=500)

model = Sequential()
model.add(Embedding(50000, 128, input_length=500))
model.add(GlobalAveragePooling1D())
model.add(Dense(90, activation='sigmoid'))

optimizer = Adam(lr=0.1)
    
model.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=[f1])

early_stop = EarlyStopping(min_delta=0.1, patience=5)

model.fit(x_train, reuters_data["y_train"],
          batch_size=64,
          epochs=10,
          callbacks=[early_stop],
          validation_data=(x_test, reuters_data["y_test"]))

Train on 7769 samples, validate on 3019 samples
Epoch 1/10
7769/7769 [==============================] - 1s 155us/step - loss: 0.0629 - f1: 0.4250 - val_loss: 0.0305 - val_f1: 0.6660
Epoch 2/10
7769/7769 [==============================] - 1s 112us/step - loss: 0.0222 - f1: 0.7559 - val_loss: 0.0223 - val_f1: 0.7589
Epoch 3/10
7769/7769 [==============================] - 1s 113us/step - loss: 0.0117 - f1: 0.8795 - val_loss: 0.0204 - val_f1: 0.7905
Epoch 4/10
7769/7769 [==============================] - 1s 115us/step - loss: 0.0061 - f1: 0.9406 - val_loss: 0.0228 - val_f1: 0.7830
Epoch 5/10
7769/7769 [==============================] - 1s 134us/step - loss: 0.0040 - f1: 0.9638 - val_loss: 0.0230 - val_f1: 0.8040
Epoch 6/10
7769/7769 [==============================] - 1s 115us/step - loss: 0.0028 - f1: 0.9754 - val_loss: 0.0232 - val_f1: 0.8177


Since I'm paranoid, I want to make sure that the model can indeed predict more than one label. I had to dig around a little to find an observation that looked nice.

In [47]:
print("Real")
print(reuters_data["y_train"][6].todense())
print("Predicted")
print(np.round(model.predict(x_train)[6], decimals=3))

Real
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]]
Predicted
[0.999 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 1.    0.    0.    0.    0.    0.   ]


### Babies

The results on the baby names is reasonable when using 2 to 5 ngrams.

In [48]:
x_train = sequence.pad_sequences(baby_data["X_train_ids"], maxlen=100)
x_test = sequence.pad_sequences(baby_data["X_test_ids"], maxlen=100)

model = Sequential()
model.add(Embedding(50000, 1, input_length=100))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=0.01)
    
model.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

early_stop = EarlyStopping(min_delta=0.01, patience=5)

model.fit(x_train, baby_data["y_train"],
          batch_size=256,
          epochs=20,
          callbacks=[early_stop],
          validation_data=(x_test, baby_data["y_test"]))

Train on 87973 samples, validate on 20000 samples
Epoch 1/20
87973/87973 [==============================] - 1s 9us/step - loss: 0.6225 - acc: 0.6462 - val_loss: 0.5967 - val_acc: 0.6331
Epoch 2/20
87973/87973 [==============================] - 1s 6us/step - loss: 0.5020 - acc: 0.7602 - val_loss: 0.5214 - val_acc: 0.7201
Epoch 3/20
87973/87973 [==============================] - 1s 6us/step - loss: 0.4372 - acc: 0.7987 - val_loss: 0.4842 - val_acc: 0.7552
Epoch 4/20
87973/87973 [==============================] - 1s 6us/step - loss: 0.4016 - acc: 0.8180 - val_loss: 0.4654 - val_acc: 0.7687
Epoch 5/20
87973/87973 [==============================] - 1s 6us/step - loss: 0.3766 - acc: 0.8311 - val_loss: 0.4663 - val_acc: 0.7622
Epoch 6/20
87973/87973 [==============================] - 1s 6us/step - loss: 0.3578 - acc: 0.8414 - val_loss: 0.4417 - val_acc: 0.7873
Epoch 7/20
87973/87973 [==============================] - 1s 6us/step - loss: 0.3434 - acc: 0.8483 - val_loss: 0.4428 - val_acc: 0.786

## Recurrent neural networks

Since the baby names are short sequences, they'll work fine with recurrent neural networks. This is based on the [example from keras](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py).

In [49]:
baby_data["X_train_ids"], baby_data["X_test_ids"] = keras_data(baby_data["X_train"], baby_data["X_test"], by_words=False, ngram_range=[1,1])

x_train = sequence.pad_sequences(baby_data["X_train_ids"], maxlen=20)
x_test = sequence.pad_sequences(baby_data["X_test_ids"], maxlen=20)

model = Sequential()
model.add(Embedding(55, 64, input_length=20))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=0.01)
    
model.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

early_stop = EarlyStopping(min_delta=0.01, patience=2)

model.fit(x_train, baby_data["y_train"],
          batch_size=64,
          epochs=10,
          callbacks=[early_stop],
          validation_data=(x_test, baby_data["y_test"]))

Size of vocabulary: 54
Train on 87973 samples, validate on 20000 samples
Epoch 1/10
87973/87973 [==============================] - 23s 264us/step - loss: 0.4550 - acc: 0.7849 - val_loss: 0.4502 - val_acc: 0.7936
Epoch 2/10
87973/87973 [==============================] - 22s 252us/step - loss: 0.4337 - acc: 0.8010 - val_loss: 0.4494 - val_acc: 0.8007
Epoch 3/10
87973/87973 [==============================] - 22s 251us/step - loss: 0.4356 - acc: 0.7984 - val_loss: 0.4562 - val_acc: 0.7948


We can run predictions on names to see how well the model does. A 1 is a predicted male name, 0 is female.

In [50]:
test_baby = ["john", "mary", "gimli", "frodo", "pikachu"]

_, test_baby_ids = keras_data(baby_data["X_train"], test_baby, by_words=False, ngram_range=[1,1])

test_baby_ids = sequence.pad_sequences(test_baby_ids, maxlen=20)

model.predict(test_baby_ids)

Size of vocabulary: 54


array([[0.8543145],
       [0.6411952],
       [0.1644226],
       [0.5717714],
       [0.283032 ]], dtype=float32)